# Natural Language Processing

# Retrieval-Augmented generation (RAG)

RAG is a technique for augmenting LLM knowledge with additional, often private or real-time, data.

LLMs can reason about wide-ranging topics, but their knowledge is limited to the public data up to a specific point in time that they were trained on. If you want to build AI applications that can reason about private data or data introduced after a model’s cutoff date, you need to augment the knowledge of the model with the specific information it needs.

<img src="../figures/RAG-process.png" >

Introducing `SwaraBot`, an innovative chatbot that answers questions about Swaraj based on his personal document.

In [2]:
import os
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

## 1. Prompt

A set of instructions or input provided by a user to guide the model's response, helping it understand the context and generate relevant and coherent language-based output, such as answering questions, completing sentences, or engaging in a conversation.

In [3]:
from langchain import PromptTemplate

prompt_template = """
    You are a helpful assistant that answers questions about the person based on their personal documents.
    Use the following context to answer the question. If you don't know the answer, just say you don't know.
    Don't make things up.    

    Context: {context}
    Question: {question}
    Answer:
    """.strip()

PROMPT = PromptTemplate.from_template(
    template = prompt_template
)

PROMPT
#using str.format 
#The placeholder is defined using curly brackets: {} {}

PromptTemplate(input_variables=['context', 'question'], template="You are a helpful assistant that answers questions about the person based on their personal documents.\n    Use the following context to answer the question. If you don't know the answer, just say you don't know.\n    Don't make things up.    \n\n    Context: {context}\n    Question: {question}\n    Answer:")

In [4]:
PROMPT.format(
    context = "This is a sample context about the person. The person has a background in computer science and has worked on various projects related to artificial intelligence.",
    question = "What is the person's background?"
)

"You are a helpful assistant that answers questions about the person based on their personal documents.\n    Use the following context to answer the question. If you don't know the answer, just say you don't know.\n    Don't make things up.    \n\n    Context: This is a sample context about the person. The person has a background in computer science and has worked on various projects related to artificial intelligence.\n    Question: What is the person's background?\n    Answer:"

## 2. Retrieval

1. `Document loaders` : Load documents from many different sources (HTML, PDF, code). 
2. `Document transformers` : One of the essential steps in document retrieval is breaking down a large document into smaller, relevant chunks to enhance the retrieval process.
3. `Text embedding models` : Embeddings capture the semantic meaning of the text, allowing you to quickly and efficiently find other pieces of text that are similar.
4. `Vector stores`: there has emerged a need for databases to support efficient storage and searching of these embeddings.
5. `Retrievers` : Once the data is in the database, you still need to retrieve it.

### 2.1 Document Loaders 
Use document loaders to load data from a source as Document's. A Document is a piece of text and associated metadata. For example, there are document loaders for loading a simple .txt file, for loading the text contents of any web page, or even for loading a transcript of a YouTube video.

In [ ]:
from langchain.document_loaders import PyMuPDFLoader

nlp_docs = '../pdfs/Swaraj.pdf'

loader = PyMuPDFLoader(nlp_docs)
documents = loader.load()

In [7]:
len(documents)

4

In [8]:
documents[1]

Document(page_content='contributed to building mass update tools that led to a notable decrease in ticket volumes. \nHis impact extended to bug ﬁxes, outbound interface conﬁgurations, and transaction \nmonitoring, making him an indispensable asset.  \nElevated to Senior Software Engineer (Jan 2024 - July 2024, Remote), he expanded his \nscope by implementing timezone-based PDF exports, developing a centralized API retry \nmechanism, and enhancing auto-reprocessing for supplier-side transactions. His \nscheduler solution for detecting and removing stuck transactions further improved \nefficiency.  \nCurrently, as an AI Engineer at AI Brain Lab (Jan 2025 - Present, On-Site, Bangkok \nMetropolitan Region, Thailand), he has successfully dockerized the frontend of the ESG \napplication, implemented CI/CD pipelines for automated deployment, and optimized \npackage management service segregation, reducing redundancy in the application. His \ncontributions have accelerated deployment processes

### 2.2 Document Transformers

This text splitter is the recommended one for generic text. It is parameterized by a list of characters. It tries to split on them in order until the chunks are small enough

In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 700,
    chunk_overlap = 100
)

doc = text_splitter.split_documents(documents)

In [10]:
doc[1]

Document(page_content='Lab while pursuing his Master’s in Data Science and Artiﬁcial Intelligence from the Asian \nInstitute of Technology (AIT), Thailand (August 2024 – May 2026). He completed his \nBachelor’s in Computer Science and Engineering from Birla Institute of Technology, Mesra, \nRanchi (2017-2021).  \n \nWork  Proﬁle \nSwaraj started his career at GEP Worldwide, where he played an instrumental role in \noptimizing internal tools, automating workﬂows, and solving over 8700 JIRA tickets. His \ntenure spanned multiple roles over 3.6 years, beginning as an intern and culminating as a \nSenior Software Engineer.  \nAs a Product Development Intern (May 2020 - July 2020, Remote), he developed a React', metadata={'source': './Swaraj.pdf', 'file_path': './Swaraj.pdf', 'page': 0, 'total_pages': 4, 'format': 'PDF 1.6', 'title': '', 'author': 'Swaraj Bhanja', 'subject': '', 'keywords': '', 'creator': 'Acrobat PDFMaker 24 for Word', 'producer': 'Adobe PDF Library 24.5.197', 'creationDat

In [11]:
len(doc)

12

### 2.3 Text Embedding Models
Embeddings create a vector representation of a piece of text. This is useful because it means we can think about text in the vector space, and do things like semantic search where we look for pieces of text that are most similar in the vector space.

In [12]:
import torch
from langchain.embeddings import HuggingFaceInstructEmbeddings

model_name = 'hkunlp/instructor-base'

embedding_model = HuggingFaceInstructEmbeddings(
    model_name = model_name,
    model_kwargs = {"device" : device}
)

/home/axiom/anaconda3/envs/NLP6/lib/python3.9/site-packages/InstructorEmbedding/instructor.py:7: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import trange


load INSTRUCTOR_Transformer
max_seq_length  512


### 2.4 Vector Stores

One of the most common ways to store and search over unstructured data is to embed it and store the resulting embedding vectors, and then at query time to embed the unstructured query and retrieve the embedding vectors that are 'most similar' to the embedded query. A vector store takes care of storing embedded data and performing vector search for you.

In [ ]:
#locate vectorstore
vector_path = '../models/vector-store-base'
if not os.path.exists(vector_path):
    os.makedirs(vector_path)
    print('create path done')

In [14]:
#save vector locally
from langchain.vectorstores import FAISS

vectordb = FAISS.from_documents(
    documents = doc,
    embedding = embedding_model
)

db_file_name = 'nlp_stanford'

vectordb.save_local(
    folder_path = os.path.join(vector_path, db_file_name),
    index_name = 'nlp' #default index
)

### 2.5 retrievers
A retriever is an interface that returns documents given an unstructured query. It is more general than a vector store. A retriever does not need to be able to store documents, only to return (or retrieve) them. Vector stores can be used as the backbone of a retriever, but there are other types of retrievers as well.

In [ ]:
#calling vector from local
vector_path = '..models/vector-store-base'
db_file_name = 'nlp_stanford'

from langchain.vectorstores import FAISS

vectordb = FAISS.load_local(
    folder_path = os.path.join(vector_path, db_file_name),
    embeddings = embedding_model,
    index_name = 'nlp' #default index
)   

In [16]:
#ready to use
retriever = vectordb.as_retriever()

In [17]:
retriever.get_relevant_documents("What is the person's background?")

[Document(page_content='Personal Aspirations \nUltimately, Swaraj’s ambition is to become a businessman, leveraging his technical \nexpertise, problem-solving mindset, and strategic vision to create impactful tech solutions.', metadata={'source': './Swaraj.pdf', 'file_path': './Swaraj.pdf', 'page': 3, 'total_pages': 4, 'format': 'PDF 1.6', 'title': '', 'author': 'Swaraj Bhanja', 'subject': '', 'keywords': '', 'creator': 'Acrobat PDFMaker 24 for Word', 'producer': 'Adobe PDF Library 24.5.197', 'creationDate': "D:20250309193041+07'00'", 'modDate': "D:20250309193043+07'00'", 'trapped': ''}),
 Document(page_content='Lab while pursuing his Master’s in Data Science and Artiﬁcial Intelligence from the Asian \nInstitute of Technology (AIT), Thailand (August 2024 – May 2026). He completed his \nBachelor’s in Computer Science and Engineering from Birla Institute of Technology, Mesra, \nRanchi (2017-2021).  \n \nWork  Proﬁle \nSwaraj started his career at GEP Worldwide, where he played an instrum

In [18]:
retriever.get_relevant_documents("What is his work experience?")

[Document(page_content='CI/CD automation. He has worked with AWS, Azure, GCP, and DigitalOcean, implementing \nzero downtime deployment strategies like Blue-Green Deployment. He is proﬁcient in \nReact, Angular, Nuxt.js (Frontend) and Django, FastAPI (Backend), with a strong focus on \nbuilding scalable web applications.  \nHis expertise in UI/UX and API development allows him to optimize and streamline system \ninteractions. His knowledge of SQL (PostgreSQL, MySQL, SQL Server), NoSQL (MongoDB, \nRedis) and Graph (GraphQL, Neo4j) is extensive. He has worked on embedding-based \noptimization in NoSQL and deployed Redis caching solutions to improve query \nperformance and data retrieval speeds.', metadata={'source': './Swaraj.pdf', 'file_path': './Swaraj.pdf', 'page': 1, 'total_pages': 4, 'format': 'PDF 1.6', 'title': '', 'author': 'Swaraj Bhanja', 'subject': '', 'keywords': '', 'creator': 'Acrobat PDFMaker 24 for Word', 'producer': 'Adobe PDF Library 24.5.197', 'creationDate': "D:202503

## 3. Memory

One of the core utility classes underpinning most (if not all) memory modules is the ChatMessageHistory class. This is a super lightweight wrapper that provides convenience methods for saving HumanMessages, AIMessages, and then fetching them all.

You may want to use this class directly if you are managing memory outside of a chain.


In [19]:
from langchain.memory import ChatMessageHistory

history = ChatMessageHistory()
history

ChatMessageHistory(messages=[])

In [20]:
history.add_user_message('hi')
history.add_ai_message('Whats up?')
history.add_user_message('How are you')
history.add_ai_message('I\'m quite good. How about you?')

In [21]:
history

ChatMessageHistory(messages=[HumanMessage(content='hi'), AIMessage(content='Whats up?'), HumanMessage(content='How are you'), AIMessage(content="I'm quite good. How about you?")])

### 3.1 Memory types

There are many different types of memory. Each has their own parameters, their own return types, and is useful in different scenarios. 
- Converstaion Buffer
- Converstaion Buffer Window

What variables get returned from memory

Before going into the chain, various variables are read from memory. These have specific names which need to align with the variables the chain expects. You can see what these variables are by calling memory.load_memory_variables({}). Note that the empty dictionary that we pass in is just a placeholder for real variables. If the memory type you are using is dependent upon the input variables, you may need to pass some in.

In this case, you can see that load_memory_variables returns a single key, history. This means that your chain (and likely your prompt) should expect an input named history. You can usually control this variable through parameters on the memory class. For example, if you want the memory variables to be returned in the key chat_history you can do:

#### Converstaion Buffer
This memory allows for storing messages and then extracts the messages in a variable.

In [22]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
memory.save_context({'input':'hi'}, {'output':'What\'s up?'})
memory.save_context({"input":'How are you?'},{'output': 'I\'m quite good. How about you?'})
memory.load_memory_variables({})

{'history': "Human: hi\nAI: What's up?\nHuman: How are you?\nAI: I'm quite good. How about you?"}

In [23]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages = True)
memory.save_context({'input':'hi'}, {'output':'What\'s up?'})
memory.save_context({"input":'How are you?'},{'output': 'I\'m quite good. How about you?'})
memory.load_memory_variables({})

{'history': [HumanMessage(content='hi'),
  AIMessage(content="What's up?"),
  HumanMessage(content='How are you?'),
  AIMessage(content="I'm quite good. How about you?")]}

#### Conversation Buffer Window
- it keeps a list of the interactions of the conversation over time. 
- it only uses the last K interactions. 
- it can be useful for keeping a sliding window of the most recent interactions, so the buffer does not get too large.

In [24]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=1)
memory.save_context({'input':'hi'}, {'output':'What\'s up?'})
memory.save_context({"input":'How are you?'},{'output': 'I\'m quite good. How about you?'})
memory.load_memory_variables({})

{'history': "Human: How are you?\nAI: I'm quite good. How about you?"}

## 4. Chain

Using an LLM in isolation is fine for simple applications, but more complex applications require chaining LLMs - either with each other or with other components.

An `LLMChain` is a simple chain that adds some functionality around language models.
- it consists of a `PromptTemplate` and a `LM` (either an LLM or chat model).
- it formats the prompt template using the input key values provided (and also memory key values, if available), 
- it passes the formatted string to LLM and returns the LLM output.

Note : [Download Fastchat Model Here](https://huggingface.co/lmsys/fastchat-t5-3b-v1.0)

In [ ]:
# %cd ../helper
# !git clone https://huggingface.co/lmsys/fastchat-t5-3b-v1.0

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Cloning into 'fastchat-t5-3b-v1.0'...
remote: Enumerating objects: 45, done.
remote: Total 45 (delta 0), reused 0 (delta 0), pack-reused 45 (from 1)
Unpacking objects: 100% (45/45), 8.84 KiB | 292.00 KiB/s, done.


In [ ]:
from transformers import AutoTokenizer, pipeline, AutoModelForSeq2SeqLM
from transformers import BitsAndBytesConfig
from langchain import HuggingFacePipeline
import torch

model_id = '../helper/fastchat-t5-3b-v1.0'

tokenizer = AutoTokenizer.from_pretrained(
    model_id)

tokenizer.pad_token_id = tokenizer.eos_token_id

bitsandbyte_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type = "nf4",
    bnb_4bit_compute_dtype = torch.float16,
    bnb_4bit_use_double_quant = True
)

model = AutoModelForSeq2SeqLM.from_pretrained(
    model_id,
    quantization_config = bitsandbyte_config, #caution Nvidia
    device_map = 'auto',
    load_in_8bit = True
)

pipe = pipeline(
    task="text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens = 256,
    model_kwargs = {
        "temperature" : 0,
        "repetition_penalty": 1.5
    }
)

llm = HuggingFacePipeline(pipeline = pipe)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARA

### [Class ConversationalRetrievalChain](https://api.python.langchain.com/en/latest/_modules/langchain/chains/conversational_retrieval/base.html#ConversationalRetrievalChain)

- `retriever` : Retriever to use to fetch documents.

- `combine_docs_chain` : The chain used to combine any retrieved documents.

- `question_generator`: The chain used to generate a new question for the sake of retrieval. This chain will take in the current question (with variable question) and any chat history (with variable chat_history) and will produce a new standalone question to be used later on.

- `return_source_documents` : Return the retrieved source documents as part of the final result.

- `get_chat_history` : An optional function to get a string of the chat history. If None is provided, will use a default.

- `return_generated_question` : Return the generated question as part of the final result.

- `response_if_no_docs_found` : If specified, the chain will return a fixed response if no docs are found for the question.


`question_generator`

In [27]:
from langchain.chains import LLMChain
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import ConversationalRetrievalChain

In [28]:
CONDENSE_QUESTION_PROMPT

PromptTemplate(input_variables=['chat_history', 'question'], template='Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.\n\nChat History:\n{chat_history}\nFollow Up Input: {question}\nStandalone question:')

In [29]:
question_generator = LLMChain(
    llm = llm,
    prompt = CONDENSE_QUESTION_PROMPT,
    verbose = True
)

In [30]:
query = 'Comparing both of them'
chat_history = "Human:What is your current country?\nAI:\nHuman:What is your previous country?\nAI:"

question_generator({'chat_history' : chat_history, "question" : query})

/home/axiom/anaconda3/envs/NLP6/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
Human:What is your current country?
AI:
Human:What is your previous country?
AI:
Follow Up Input: Comparing both of them
Standalone question:

> Finished chain.


{'chat_history': 'Human:What is your current country?\nAI:\nHuman:What is your previous country?\nAI:',
 'question': 'Comparing both of them',
 'text': '<pad> What  is  the  difference  between  your  current  and  previous  country?\n'}

`combine_docs_chain`

In [31]:
doc_chain = load_qa_chain(
    llm = llm,
    chain_type = 'stuff',
    prompt = PROMPT,
    verbose = True
)
doc_chain

StuffDocumentsChain(verbose=True, llm_chain=LLMChain(verbose=True, prompt=PromptTemplate(input_variables=['context', 'question'], template="You are a helpful assistant that answers questions about the person based on their personal documents.\n    Use the following context to answer the question. If you don't know the answer, just say you don't know.\n    Don't make things up.    \n\n    Context: {context}\n    Question: {question}\n    Answer:"), llm=HuggingFacePipeline(pipeline=<transformers.pipelines.text2text_generation.Text2TextGenerationPipeline object at 0x779950ead490>)), document_variable_name='context')

In [32]:
query = "What is your current job?"
input_document = retriever.get_relevant_documents(query)

doc_chain({'input_documents':input_document, 'question':query})



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
You are a helpful assistant that answers questions about the person based on their personal documents.
    Use the following context to answer the question. If you don't know the answer, just say you don't know.
    Don't make things up.    

    Context: failures. 
Upon transitioning into a full-time Software Engineer (July 2021 - Dec 2023, Remote), he 
played a key role in standardizing HTML templates for transactional documents, developing 
bulk APIs that reduced data correction ticket resolution times from hours to minutes, and 
enhancing cXML/JSON special character handling to eliminate interface failures. He 
automated transaction reprocessing, signiﬁcantly reducing manual interventions, and

Lab while pursuing his Master’s in Data Science and Artiﬁcial Intelligence from the Asian 
Institute of Technology (AIT), Thailand (August 2024 – May 2026). He completed his 
Bachelor’s 

{'input_documents': [Document(page_content='failures. \nUpon transitioning into a full-time Software Engineer (July 2021 - Dec 2023, Remote), he \nplayed a key role in standardizing HTML templates for transactional documents, developing \nbulk APIs that reduced data correction ticket resolution times from hours to minutes, and \nenhancing cXML/JSON special character handling to eliminate interface failures. He \nautomated transaction reprocessing, signiﬁcantly reducing manual interventions, and', metadata={'source': './Swaraj.pdf', 'file_path': './Swaraj.pdf', 'page': 0, 'total_pages': 4, 'format': 'PDF 1.6', 'title': '', 'author': 'Swaraj Bhanja', 'subject': '', 'keywords': '', 'creator': 'Acrobat PDFMaker 24 for Word', 'producer': 'Adobe PDF Library 24.5.197', 'creationDate': "D:20250309193041+07'00'", 'modDate': "D:20250309193043+07'00'", 'trapped': ''}),
  Document(page_content='Lab while pursuing his Master’s in Data Science and Artiﬁcial Intelligence from the Asian \nInstitute of

In [33]:
memory = ConversationBufferWindowMemory(
    k=3, 
    memory_key = "chat_history",
    return_messages = True,
    output_key = 'answer'
)

chain = ConversationalRetrievalChain(
    retriever=retriever,
    question_generator=question_generator,
    combine_docs_chain=doc_chain,
    return_source_documents=True,
    memory=memory,
    verbose=True,
    get_chat_history=lambda h : h
)
chain

ConversationalRetrievalChain(memory=ConversationBufferWindowMemory(output_key='answer', return_messages=True, memory_key='chat_history', k=3), verbose=True, combine_docs_chain=StuffDocumentsChain(verbose=True, llm_chain=LLMChain(verbose=True, prompt=PromptTemplate(input_variables=['context', 'question'], template="You are a helpful assistant that answers questions about the person based on their personal documents.\n    Use the following context to answer the question. If you don't know the answer, just say you don't know.\n    Don't make things up.    \n\n    Context: {context}\n    Question: {question}\n    Answer:"), llm=HuggingFacePipeline(pipeline=<transformers.pipelines.text2text_generation.Text2TextGenerationPipeline object at 0x779950ead490>)), document_variable_name='context'), question_generator=LLMChain(verbose=True, prompt=PromptTemplate(input_variables=['chat_history', 'question'], template='Given the following conversation and a follow up question, rephrase the follow up 

## 5. Chatbot

In [34]:
prompt_question = "Who are you by the way?"
answer = chain({"question":prompt_question})
answer



> Entering new ConversationalRetrievalChain chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
You are a helpful assistant that answers questions about the person based on their personal documents.
    Use the following context to answer the question. If you don't know the answer, just say you don't know.
    Don't make things up.    

    Context: Introduction 
Swaraj Bhanja is a highly skilled Cloud Engineer, AI Enthusiast, and Technologist with 
expertise spanning cloud computing, DevOps, full-stack development, machine learning, 
and AI infrastructure optimization. His journey has been deﬁned by deep technical 
expertise, an insatiable curiosity for problem-solving, and a relentless drive for efficiency.  
 
Personal Information 
Born on October 21, 1997, in Jamshedpur, Jharkhand, India, he hails from Kochi, Kerala, 
and is currently based in Bangkok, Thailand, where he works as an AI Engineer at AI Brain 
Lab while 

{'question': 'Who are you by the way?',
 'chat_history': [],
 'answer': '<pad>  I  am  an  AI  assistant  and  do  not  have  personal  information.\n',
 'source_documents': [Document(page_content='Introduction \nSwaraj Bhanja is a highly skilled Cloud Engineer, AI Enthusiast, and Technologist with \nexpertise spanning cloud computing, DevOps, full-stack development, machine learning, \nand AI infrastructure optimization. His journey has been deﬁned by deep technical \nexpertise, an insatiable curiosity for problem-solving, and a relentless drive for efficiency.  \n \nPersonal Information \nBorn on October 21, 1997, in Jamshedpur, Jharkhand, India, he hails from Kochi, Kerala, \nand is currently based in Bangkok, Thailand, where he works as an AI Engineer at AI Brain \nLab while pursuing his Master’s in Data Science and Artiﬁcial Intelligence from the Asian', metadata={'source': './Swaraj.pdf', 'file_path': './Swaraj.pdf', 'page': 0, 'total_pages': 4, 'format': 'PDF 1.6', 'title': '', 

In [35]:
prompt_question = "What is your technology stack?"
answer = chain({"question":prompt_question})
answer



> Entering new ConversationalRetrievalChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
[HumanMessage(content='Who are you by the way?'), AIMessage(content='<pad>  I  am  an  AI  assistant  and  do  not  have  personal  information.\n')]
Follow Up Input: What is your technology stack?
Standalone question:

> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
You are a helpful assistant that answers questions about the person based on their personal documents.
    Use the following context to answer the question. If you don't know the answer, just say you don't know.
    Don't make things up.    

    Context: Notable Projects & Contributions 
1. Multi-Cloud Web Application: Hosted across AWS, Azure, and DigitalOcean, 
featuring Kub

{'question': 'What is your technology stack?',
 'chat_history': [HumanMessage(content='Who are you by the way?'),
  AIMessage(content='<pad>  I  am  an  AI  assistant  and  do  not  have  personal  information.\n')],
 'answer': '<pad>  React,  Angular,  Nuxt.js  (Frontend)  and  Django,  FastAPI  (Backend).\n',
 'source_documents': [Document(page_content='Notable Projects & Contributions \n1. Multi-Cloud Web Application: Hosted across AWS, Azure, and DigitalOcean, \nfeaturing Kubernetes-based load balancing and zero downtime deployment. \n2. Web Version of Ubuntu 18.04 (Hackathon Project, 2018): Created a web-based \nUbuntu interface as part of a team of 4. \n3. Custom VPN on Raspberry Pi: Set up a secure VPN that allowed international \naccess to a home network. \n4. Automated Data Processing Pipelines: Developed mass data correction and auto-\nreprocessing tools, reducing operational overhead. \n \nPersonal Interests & Philosophy \nSwaraj views experience through the lens of dynamic 

In [36]:
prompt_question = "Are you familiar with Cloud Computing?"
answer = chain({"question":prompt_question})
answer



> Entering new ConversationalRetrievalChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
[HumanMessage(content='Who are you by the way?'), AIMessage(content='<pad>  I  am  an  AI  assistant  and  do  not  have  personal  information.\n'), HumanMessage(content='What is your technology stack?'), AIMessage(content='<pad>  React,  Angular,  Nuxt.js  (Frontend)  and  Django,  FastAPI  (Backend).\n')]
Follow Up Input: Are you familiar with Cloud Computing?
Standalone question:

> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
You are a helpful assistant that answers questions about the person based on their personal documents.
    Use the following context to answer the question. If you don't know the answer, just say you don't know.


{'question': 'Are you familiar with Cloud Computing?',
 'chat_history': [HumanMessage(content='Who are you by the way?'),
  AIMessage(content='<pad>  I  am  an  AI  assistant  and  do  not  have  personal  information.\n'),
  HumanMessage(content='What is your technology stack?'),
  AIMessage(content='<pad>  React,  Angular,  Nuxt.js  (Frontend)  and  Django,  FastAPI  (Backend).\n')],
 'answer': '<pad>  It  is  a  type  of  computing  where  organizations  rent  computing  resources  from  a  third-party  provider  such  as  Amazon  Web  Services  (AWS),  Microsoft  Azure,  Google  Cloud  Platform  (GCP),  or  DigitalOcean.  The  provider  manages  the  infrastructure  and  provides  the  resources  needed  to  run  the  application.  This  allows  organizations  to  focus  on  their  core  business  while  the  provider  handles  the  infrastructure  and  maintenance.\n',
 'source_documents': [Document(page_content='Swaraj is captivated by Christopher Nolan’s ﬁlms, with Interstellar,